In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from methods_of_Dcalc import Dpointder, Dlinalg, Dlinalgw

import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
#Currently, my code only has capability to do frame-by-frame analysis on 1 video.  Perhaps future functionality will
#be able to expand beyond this.  

logplot = 'confocal_625ms_60x_1_logplot'
Mplot = 'confocal_625ms_60x_1_Mplot'
Dplot = 'confocal_625ms_60x_1_Dplot'
Hplot = 'confocal_625ms_60x_1_Hplot'
Hlogplot = 'confocal_625ms_60x_1_Hlogplot'
Cplot = 'confocal_625ms_60x_1_Cplot'

conversion = dict() #First element is the umppx, second is fps

conversion[1] = (0.11, 1.62, 1)
# conversion[2] = (0.30, 5.37, 1)
# conversion[3] = (0.30, 5.37, 1)
# conversion[4] = (0.30, 5.31, 1)
# conversion[5] = (0.30, 5.33, 1)
# conversion[6] = (0.30, 5.33, 1)
# conversion[7] = (0.30, 5.35, 1)

trajectory = dict()
cut = 5

trajectory[1] = np.genfromtxt('./confocal_625ms_60x_1.csv',
            delimiter =",")
# trajectory[2] = np.genfromtxt('./CON_40x_2D_130ms_2_T.csv',
#             delimiter =",")
# trajectory[3] = np.genfromtxt('./CON_40x_2D_130ms_3_T.csv',
#             delimiter =",")
# trajectory[4] = np.genfromtxt('./CON_40x_2D_130ms_4_T.csv',
#             delimiter =",")
# trajectory[5] = np.genfromtxt('./CON_40x_2D_130ms_5_T.csv',
#             delimiter =",")
# trajectory[6] = np.genfromtxt('./CON_40x_2D_130ms_6_T.csv',
#             delimiter =",")
# trajectory[7] = np.genfromtxt('./CON_40x_2D_130ms_7_T.csv',
#             delimiter =",")

totvids = 1

for num in range(1, totvids + 1):
    
    #Remove titles from columns
    #trajectory[num]=np.delete(trajectory[num], 0,0)
    #Remove the number column from dataset
    trajectory[num]=np.delete(trajectory[num],0,1)


In [3]:
def fillin2(data):
    """
    Fills in blanks of arrays without shifting frames by the starting frame.  Compare to fillin.
    """
    shap = int(max(data[:, 1])) + 1
    shape1 = int(min(data[:, 1]))
    newshap = shap - shape1
    filledin = np.zeros((newshap, 5))
    filledin[0, :] = data[0, :]

    count = 0
    new = 0
    other = 0
    tot = 0

    for num in range(1, newshap):
        if data[num-new, 1]-data[num-new-1, 1] == 1:
            count = count + 1
            filledin[num, :] = data[num-new, :]
        elif data[num - new, 1]-data[num - new - 1, 1] > 1:
            new = new + 1
            iba = int(data[num - new+1, 1]-data[num - new, 1])
            togoin = data[num - new]
            togoin[1] = togoin[1] + 1
            filledin[num, :] = togoin
            # dataset[2] = np.insert(dataset[2], [num + new - 2], togoin, axis=0)

        else:
            other = other + 1
        tot = count + new + other

    return filledin

In [4]:
rawframes = trajectory[1][:, 1]
#frames = np.linspace()
frames = np.linspace(min(rawframes),max(rawframes), max(rawframes)+1).astype(np.int64)
time = frames / conversion[1][1]

x = dict()
y = dict()
z = dict()
M1x = dict()
M1y = dict()
M1z = dict()
M2xy = dict()
M2xz = dict()
M2yz = dict()
M3 = dict()

In [5]:
dataset = dict()

particles = trajectory[1][:, 0]
total = int(max(particles))
total1 = total + 1
rawdataset = trajectory[1][:, :]

#Perform conversion on raw dataset
rawdataset[:, 2:4] = conversion[1][0] * rawdataset[:, 2:4]
rawdataset[:, 4] = conversion[1][2] * rawdataset[:, 4]

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    dataset[num] = (rawdataset[min1:max1+1, 0:5])

In [6]:
#I is a set of all nonzero elements in trajectories.
I = dict()
for num in range(1, total1):
    #Construct x, y, z
    dataset[num] = fillin2(dataset[num])
    x[num] = np.zeros(frames.shape[0])
    x[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 2]
    y[num] = np.zeros(frames.shape[0])
    y[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 3]
    z[num] = np.zeros(frames.shape[0])
    z[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 4]

In [7]:
np.count_nonzero(x[300])

2

In [8]:
total1

318

In [9]:
#I wrote a short bit of code to filter out trajectories that are too short to be included.  This also reduces computing
#time by cutting down on MSDs that need to be calculated.  
cut = 5
cutoff = cut
x1 = dict()
y1 = dict()
z1 = dict()
fifties = 0
nones = 0

for num in range(1, total1):
    if np.count_nonzero(x[num]) < cutoff:
        nones = nones + 1
    else:
        fifties = fifties + 1
        x1[num - nones] = x[num]
        y1[num - nones] = y[num]
        z1[num - nones] = z[num]
        # I must also redefine the particle numbers to reflect the new set.
        #teancum[num - nones][:, 0] = fifties

x = x1
y = y1
z = z1

In [10]:
#fifties is the new total1 after filtering out short trajectories
total1 = fifties + 1
fifties

238

In [11]:
for num in range(1, total1):

    #Construct MSD and Deff dictionaries
    M1x[num] = np.zeros(frames.shape[0])
    M1x[num][0:frames.shape[0]] = 0
    M1y[num] = np.zeros(frames.shape[0])
    M1y[num][0:frames.shape[0]] = 0
    M1z[num] = np.zeros(frames.shape[0])
    M1z[num][0:frames.shape[0]] = 0
    M2xy[num] = np.zeros(frames.shape[0])
    M2xy[num][0:frames.shape[0]] = 0
    M2xz[num] = np.zeros(frames.shape[0])
    M2xz[num][0:frames.shape[0]] = 0
    M2yz[num] = np.zeros(frames.shape[0])
    M2yz[num][0:frames.shape[0]] = 0
    M3[num] = np.zeros(frames.shape[0])
    M3[num][0:frames.shape[0]] = 0
    
    #Fill in MSD and Deff dictionaries
    I[num] = np.nonzero(x[num])[0]
    first = I[num][0]
    last = I[num][-1] + 1
    startx = x[num][first]
    starty = y[num][first]
    startz = z[num][first]
    
    for num1 in range(first, last):
        M1x[num][num1] = (x[num][num1] - startx)**2
        M1y[num][num1] = (y[num][num1] - starty)**2
        M1z[num][num1] = (z[num][num1] - startz)**2
        M2xy[num][num1] = (x[num][num1] - startx)**2 + (y[num][num1] - starty)**2
        M2xz[num][num1] = (x[num][num1] - startx)**2 + (z[num][num1] - startz)**2
        M2yz[num][num1] = (z[num][num1] - startz)**2 + (y[num][num1] - starty)**2
        M3[num][num1] = (z[num][num1] - startz)**2 + (y[num][num1] - starty)**2 + (x[num][num1] - startx)**2
        
#         M1x[num][first] = 'NaN'
#         M1y[num][first] = 'NaN'
#         M1z[num][first] = 'NaN'
#         M2xy[num][first] = 'NaN'
#         M2xz[num][first] = 'NaN'
#         M2yz[num][first] = 'NaN'
#         M3[num][first] = 'NaN'
    M1x[num] = ma.masked_equal(M1x[num], 0)
    M1y[num] = ma.masked_equal(M1y[num], 0)
    M1z[num] = ma.masked_equal(M1z[num], 0)
    M2xy[num] = ma.masked_equal(M2xy[num], 0)
    M2xz[num] = ma.masked_equal(M2xz[num], 0)
    M2yz[num] = ma.masked_equal(M2yz[num], 0)
    M3[num] = ma.masked_equal(M3[num], 0)

In [12]:
#np.count_nonzero(M1x[495])
shift = next((i for i, x in enumerate(M1x[50]) if x), None)
sM1x = dict()
sM1x[50] = np.zeros(frames.shape[0] - shift)
for num in range(0, M1x[50].shape[0] - shift):
    sM1x[50][num] = M1x[50][num + shift]
sM1x

{50: array([  1.23904000e-05,   1.16281000e-05,   4.84000000e-06,
          4.65124000e-05,   9.80100000e-07,   1.08900000e-07,
          5.92900000e-05,   4.95616000e-05,   4.84000000e-08,
          1.51782400e-04,   9.80100000e-07,   2.56036000e-05,
          1.60022500e-04,   5.11225000e-05,   6.27264000e-05,
          1.48225000e-05,   8.33569000e-05,   9.80100000e-05,
          2.45025000e-05,   1.74724000e-05,   9.37024000e-05,
          1.33402500e-04,   2.01356100e-04,   4.35600000e-05,
          1.54504900e-04,   1.46410000e-04,   6.27264000e-05,
          2.68632100e-04,   9.80100000e-05,   1.35955600e-04,
          1.65636900e-04,   1.02414400e-04,   7.55161000e-05,
          2.04490000e-04,   6.80625000e-05,   1.30873600e-04,
          8.33569000e-05,   1.77156100e-04,   7.17409000e-05,
          4.80249000e-05,   2.78784000e-05,   8.53776000e-05,
          5.76081000e-05,   8.94916000e-05,   1.57251600e-04,
          4.21201000e-05,   1.00200100e-04,   2.01356100e-04,
    

In [13]:
M1x[50]

masked_array(data = [-- 1.2390400000012278e-05 1.1628100000016065e-05 4.8399999999930715e-06
 4.65124000000158e-05 9.80099999996135e-07 1.0890000000113376e-07
 5.92899999999972e-05 4.956159999999909e-05 4.840000000102495e-08
 0.00015178239999997533 9.80099999996135e-07 2.56036000000029e-05
 0.00016002250000001812 5.112249999999033e-05 6.272640000003401e-05
 1.48224999999993e-05 8.335689999998123e-05 9.801000000003557e-05
 2.4502500000008892e-05 1.74724000000136e-05 9.370239999998968e-05
 0.00013340250000007576 0.00020135609999997897 4.3559999999984546e-05
 0.0001545048999999599 0.00014641000000000537 6.272640000003401e-05
 0.00026863210000004057 9.801000000003557e-05 0.00013595559999997938
 0.00016563689999998696 0.0001024144000000116 7.551610000002466e-05
 0.0002044899999999613 6.806250000000516e-05 0.00013087360000000776
 8.335689999998123e-05 0.00017715610000001595 7.174089999998457e-05
 4.802490000000758e-05 2.7878399999990107e-05 8.537760000003535e-05
 5.760810000000652e-05 8.9491

In [14]:
#Each entry in yes is an array of particles that are in the given frame.  The frame is given by the index of yes.
yes = dict()
yessize = dict()
for num1 in range(1, frames.shape[0]):
    count = 0
    yes[num1] = np.zeros(total1).astype(np.int64)
    for num in range(1, total1):
        check = num1 in I[num]
        if check == True:
            yes[num1][count] = num
            count = count + 1
    iindex = np.where(yes[num1]==max(yes[num1]))[0][0]
    yes[num1] = yes[num1][0:iindex]

In [15]:
#And the FBF analyis
framed = dict()
fM1y = dict()
fM1z = dict()
fM2xy = dict()
fM2xz = dict()
fM2yz = dict()
fM3 = dict()

for num2 in range(1, frames.shape[0]):
    framed[num2] = np.zeros(frames.shape[0])
    fM1y[num2] = np.zeros(frames.shape[0])
    fM1z[num2] = np.zeros(frames.shape[0])
    fM2xy[num2] = np.zeros(frames.shape[0])
    fM2xz[num2] = np.zeros(frames.shape[0])
    fM2yz[num2] = np.zeros(frames.shape[0])
    fM3[num2] = np.zeros(frames.shape[0])
    
    for num1 in range(0, frames.shape[0]):
        frame = np.zeros(yes[num2].shape[0])
        fmM1y = np.zeros(yes[num2].shape[0])
        fmM1z = np.zeros(yes[num2].shape[0])
        fmM2xy = np.zeros(yes[num2].shape[0])
        fmM2xz = np.zeros(yes[num2].shape[0])
        fmM2yz = np.zeros(yes[num2].shape[0])
        fmM3 = np.zeros(yes[num2].shape[0])
        
        for num in range(0, yes[num2].shape[0]):
            frame[num] = M1x[yes[num2][num]][num1]
            fmM1y[num] = M1y[yes[num2][num]][num1]
            fmM1z[num] = M1z[yes[num2][num]][num1]
            fmM2xy[num] = M2xy[yes[num2][num]][num1]
            fmM2xz[num] = M2xz[yes[num2][num]][num1]
            fmM2yz[num] = M2yz[yes[num2][num]][num1]
            fmM3[num] = M3[yes[num2][num]][num1]
        frame = ma.masked_invalid(frame)
        fmM1y = ma.masked_invalid(fmM1y)
        fmM1z = ma.masked_invalid(fmM1z)
        fmM2xy = ma.masked_invalid(fmM2xy)
        fmM2xz = ma.masked_invalid(fmM2xz)
        fmM2yz = ma.masked_invalid(fmM2yz)
        fmM3 = ma.masked_invalid(fmM3)
        
        framed[num2][num1] = stat.gmean(frame)
        fM1y[num2][num1] = stat.gmean(fmM1y)
        fM1z[num2][num1] = stat.gmean(fmM1z)
        fM2xy[num2][num1] = stat.gmean(fmM2xy)
        fM2xz[num2][num1] = stat.gmean(fmM2xz)
        fM2yz[num2][num1] = stat.gmean(fmM2yz)
        fM3[num2][num1] = stat.gmean(fmM3)

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [16]:
FM1x = np.zeros(frames.shape[0])
FM1y = np.zeros(frames.shape[0])
FM1z = np.zeros(frames.shape[0])
FM2xy = np.zeros(frames.shape[0])
FM2xz = np.zeros(frames.shape[0])
FM2yz = np.zeros(frames.shape[0])
FM3 = np.zeros(frames.shape[0])

for num in range(1, frames.shape[0]):
    FM1x = FM1x + framed[num]
    FM1y = FM1y + fM1y[num]
    FM1z = FM1z + fM1z[num]
    FM2xy = FM2xy + fM2xy[num]
    FM2xz = FM2xz + fM2xz[num]
    FM2yz = FM2yz + fM2yz[num]
    FM3 = FM3 + fM3[num]
    
FM1x = FM1x/frames.shape[0]
FM1y = FM1y/frames.shape[0]
FM1z = FM1z/frames.shape[0]
FM2xy = FM2xy/frames.shape[0]
FM2xz = FM2xz/frames.shape[0]
FM2yz = FM2yz/frames.shape[0]
FM3 = FM3/frames.shape[0]



In [17]:
FD1x = FM1x/time
FD1y = FM1y/time
FD1z = FM1z/time
FD2xy = FM2xy/time
FD2xz = FM2xz/time
FD2yz = FM2yz/time
FD3 = FM3/time

In [18]:
def plot_MLOG(M2xy, Mx, My, time, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    M2 = M2xy
    Mx = Mx
    My = My
    t = time
    
    #I will need to calculate standard deviations eventually
#     #Now to calculate the standard dev at each point:
#     for num in range (1, total1):
#         SDunit = (MSD[num] - MMSD)**2
#         SD = SD + SDunit
#     SD = np.sqrt(SD/total1)
#     SE = SD/np.sqrt(total1)
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, M2, linestyle='', linewidth=10, label='2D', marker='o', ms=10, color='blue')
    ax.plot(t, Mx, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='red')
    ax.plot(t, My, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='green')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
    
#     ax.plot(t, linel[0], linewidth=3, color='blue')
#     ax.plot(t, linel[1], linewidth=3, color='red')
#     ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.76), prop={'size': 40})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    plt.yscale('log')
    plt.xscale('log')
    plt.gca().set_xlim([0.1, limit1])
    plt.gca().set_ylim([0.00001, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return M2, Mx

In [19]:
one1, bleh = plot_MLOG(FM2xy, FM1x, FM1y, time, 1, 1, 'MSD (um^2)', logplot, 100.1, 0.01, 1, 1)
plt.show()

In [20]:
def plot_M(M2xy, Mx, My, time, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    M2 = M2xy
    Mx = Mx
    My = My
    t = time
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, M2, linestyle='', linewidth=10, label='2D', marker='o', ms=10, color='blue')
    ax.plot(t, Mx, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='red')
    ax.plot(t, My, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='green')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
#     ax.plot(t, line[0], linewidth=3, color='blue')
#     ax.plot(t, line[1], linewidth=3, color='red')
#     ax.plot(t, line[2], linewidth=3, color='green')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.76), prop={'size': 40})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return M2, Mx

In [21]:
one1, bleh = plot_M(FM2xy, FM1x, FM1y, time, 0, 4, 'MSD (um^2)', Mplot, 50.1, 0.001, 10, 0.0002)
plt.show()

In [22]:
def plot_M1(M2xy, Mx, My, time, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    M2 = M2xy
    Mx = Mx
    My = My
    t = time
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, M2, linewidth=10, label='2D', color='blue')
    ax.plot(t, Mx, linewidth=10, label='1D x', color='red')
    ax.plot(t, My, linewidth=10, label='1D x', color='green')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
#     ax.plot(t, line[0], linewidth=3, color='blue')
#     ax.plot(t, line[1], linewidth=3, color='red')
#     ax.plot(t, line[2], linewidth=3, color='green')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.76), prop={'size': 40})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return M2, Mx

In [23]:
one1, bleh = plot_M1(FD2xy, FD1x, FD1y, time, 0, 5, 'MSD (um^2)', Mplot, 50.1, 0.00011, 10, 0.00002)
plt.show()

In [24]:
#This block will create dictionaries of shifted trajectories to calculate arithmetic and geometric means.  I need to so
#I can compare FBF analysis with them.

final, tots = prettify(trajectory[1], 50, 50, conversion[1][0], conversion[1][1], conversion[1][2])

SM1x = dict()
SM1y = dict()
SM1z = dict()
SM2xy = dict()
SM2xz = dict()
SM2yz = dict()
SM3 = dict()

SD1x = dict()
SD1y = dict()
SD1z = dict()
SD2xy = dict()
SD2xz = dict()
SD2yz = dict()
SD3 = dict()

time2 = final[1][:, 15]

for num in range(1, tots + 1):
    SM1x[num] = final[num][:, 12]
    SM1y[num] = final[num][:, 13]
    SM1z[num] = final[num][:, 14]
    SM2xy[num] = final[num][:, 9]
    SM2xz[num] = final[num][:, 10]
    SM2yz[num] = final[num][:, 11]
    SM3[num] = final[num][:, 8]

    SD1x[num] = final[num][:, 20]
    SD1y[num] = final[num][:, 21]
    SD1z[num] = final[num][:, 22]
    SD2xy[num] = final[num][:, 17]
    SD2xz[num] = final[num][:, 18]
    SD2yz[num] = final[num][:, 19]
    SD3[num] = final[num][:, 16]

In [25]:
tots

167

In [26]:
#Now to calculate arithmetic and geometric means.

arM1x = np.zeros(SM1x[1].shape[0])
arM1y = np.zeros(SM1x[1].shape[0])
arM1z = np.zeros(SM1x[1].shape[0])
arM2xy = np.zeros(SM1x[1].shape[0])
arM2xz = np.zeros(SM1x[1].shape[0])
arM2yz = np.zeros(SM1x[1].shape[0])
arM3 = np.zeros(SM1x[1].shape[0])

arD1x = np.zeros(SM1x[1].shape[0])
arD1y = np.zeros(SM1x[1].shape[0])
arD1z = np.zeros(SM1x[1].shape[0])
arD2xy = np.zeros(SM1x[1].shape[0])
arD2xz = np.zeros(SM1x[1].shape[0])
arD2yz = np.zeros(SM1x[1].shape[0])
arD3 = np.zeros(SM1x[1].shape[0])

for num in range(1, tots +1):
    arM1x = arM1x + SM1x[num]
    arM1y = arM1y + SM1y[num]
    arM1z = arM1z + SM1z[num]
    arM2xy = arM2xy + SM2xy[num]
    arM2xz = arM2xz + SM2xz[num]
    arM2yz = arM2yz + SM2yz[num]
    arM3 = arM3 + SM3[num]
    
    arD1x = arD1x + SD1x[num]
    arD1y = arD1y + SD1y[num]
    arD1z = arD1z + SD1z[num]
    arD2xy = arD2xy + SD2xy[num]
    arD2xz = arD2xz + SD2xz[num]
    arD2yz = arD2yz + SD2yz[num]
    arD3 = arD3 + SD3[num]
    
arM1x = arM1x/tots
arM1y = arM1y/tots
arM1z = arM1z/tots
arM2xy = arM2xy/tots
arM2xz = arM2xz/tots
arM2yz = arM2yz/tots
arM3 = arM3/tots

arD1x = arD1x/tots
arD1y = arD1y/tots
arD1z = arD1z/tots
arD2xy = arD2xy/tots
arD2xz = arD2xz/tots
arD2yz = arD2yz/tots
arD3 = arD3/tots

In [27]:
gM1x = dict()
gM1y = dict()
gM1z = dict()
gM2xy = dict()
gM2xz = dict()
gM2yz = dict()
gM3 = dict()

gD1x = dict()
gD1y = dict()
gD1z = dict()
gD2xy = dict()
gD2xz = dict()
gD2yz = dict()
gD3 = dict()

geoM1x = np.zeros(SM1x[1].shape[0])
geoM1y = np.zeros(SM1x[1].shape[0])
geoM1z = np.zeros(SM1x[1].shape[0])
geoM2xy = np.zeros(SM1x[1].shape[0])
geoM2xz = np.zeros(SM1x[1].shape[0])
geoM2yz = np.zeros(SM1x[1].shape[0])
geoM3 = np.zeros(SM1x[1].shape[0])

geoD1x = np.zeros(SM1x[1].shape[0])
geoD1y = np.zeros(SM1x[1].shape[0])
geoD1z = np.zeros(SM1x[1].shape[0])
geoD2xy = np.zeros(SM1x[1].shape[0])
geoD2xz = np.zeros(SM1x[1].shape[0])
geoD2yz = np.zeros(SM1x[1].shape[0])
geoD3 = np.zeros(SM1x[1].shape[0])

for num2 in range(1, SM1x[1].shape[0] + 1):
    gM1x[num2] = np.zeros(tots)
    gM1y[num2] = np.zeros(tots)
    gM1z[num2] = np.zeros(tots)
    gM2xy[num2] = np.zeros(tots)
    gM2xz[num2] = np.zeros(tots)
    gM2yz[num2] = np.zeros(tots)
    gM3[num2] = np.zeros(tots)
    
    gD1x[num2] = np.zeros(tots)
    gD1y[num2] = np.zeros(tots)
    gD1z[num2] = np.zeros(tots)
    gD2xy[num2] = np.zeros(tots)
    gD2xz[num2] = np.zeros(tots)
    gD2yz[num2] = np.zeros(tots)
    gD3[num2] = np.zeros(tots)
    
    for num in range(1, tots + 1):
        gM1x[num2][num-1] = SM1x[num][num2-1]
        gM1y[num2][num-1] = SM1y[num][num2-1]
        gM1z[num2][num-1] = SM1z[num][num2-1]
        gM2xy[num2][num-1] = SM2xy[num][num2-1]
        gM2xz[num2][num-1] = SM2xz[num][num2-1]
        gM2yz[num2][num-1] = SM2yz[num][num2-1]
        gM3[num2][num-1] = SM3[num][num2-1]
        
        gD1x[num2][num-1] = SD1x[num][num2-1]
        gD1y[num2][num-1] = SD1y[num][num2-1]
        gD1z[num2][num-1] = SD1z[num][num2-1]
        gD2xy[num2][num-1] = SD2xy[num][num2-1]
        gD2xz[num2][num-1] = SD2xz[num][num2-1]
        gD2yz[num2][num-1] = SD2yz[num][num2-1]
        gD3[num2][num-1] = SD3[num][num2-1]
        
    geoM1x[num2 - 1] = stat.gmean(gM1x[num2])
    geoM1y[num2 - 1] = stat.gmean(gM1y[num2])
    geoM1z[num2 - 1] = stat.gmean(gM1z[num2])
    geoM2xy[num2 - 1] = stat.gmean(gM2xy[num2])
    geoM2xz[num2 - 1] = stat.gmean(gM2xz[num2])
    geoM2yz[num2 - 1] = stat.gmean(gM2yz[num2])
    geoM3[num2 - 1] = stat.gmean(gM3[num2])

    geoD1x[num2 - 1] = stat.gmean(gD1x[num2])
    geoD1y[num2 - 1] = stat.gmean(gD1y[num2])
    geoD1z[num2 - 1] = stat.gmean(gD1z[num2])
    geoD2xy[num2 - 1] = stat.gmean(gD2xy[num2])
    geoD2xz[num2 - 1] = stat.gmean(gD2xz[num2])
    geoD2yz[num2 - 1] = stat.gmean(gD2yz[num2])
    geoD3[num2 - 1] = stat.gmean(gD3[num2])

In [28]:
geoD1x

array([  0.00000000e+00,   2.42522627e-07,   0.00000000e+00,
         5.54772374e-08,   6.58356250e-08,   0.00000000e+00,
         3.55429630e-08,   3.87613061e-08,   3.55739314e-08,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   3.55732822e-08,   0.00000000e+00,
         0.00000000e+00,   3.67620726e-08,   0.00000000e+00,
         0.00000000e+00,   2.83922645e-08,   3.57755785e-08,
         3.98422742e-08,   3.67803371e-08,   3.10193371e-08,
         0.00000000e+00,   2.70885821e-08,   0.00000000e+00,
         4.28382539e-08,   4.01805974e-08,   3.11010578e-08,
         3.84008929e-08,   4.64282234e-08,   3.95642928e-08,
         4.76466900e-08,   4.31671987e-08,   3.09696349e-08,
         4.45659030e-08,   3.66023390e-08,   3.96543843e-08,
         4.09095563e-08,   0.00000000e+00,   0.00000000e+00,
         3.64605312e-08,   4.40256098e-08,   3.95028286e-08,
         4.25556882e-08,   3.69961794e-08,   3.99500726e-08,
         3.70860233e-08,

In [29]:
def plot_Mcomp(Mgeo, Mar, MFBF, t1, t2, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Mgeo = Geometric average dataset
    Mar = Arithmetic average dataset
    MFBF = Frame-by-Frame dataset
    
    t1 = time array for Mgeo and Mar
    t2 time array for MFDF
    """
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t1, Mgeo, linewidth=10, label='Arithmetic', color='blue')
    ax.plot(t1, Mar, linewidth=10, label='Geometric', color='red')
    ax.plot(t2[4:-1], MFBF[4:-1], linewidth=10, label='Frame-by-Frame', color='green')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
#     ax.plot(t, line[0], linewidth=3, color='blue')
#     ax.plot(t, line[1], linewidth=3, color='red')
#     ax.plot(t, line[2], linewidth=3, color='green')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.76), prop={'size': 40})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return Mar, Mgeo

In [33]:
one1, bleh = plot_Mcomp(geoD2xy, arD2xy, FD2xy, time2, time, 0, 5, 'Deff (um^2/s)', Cplot, 50.1, 0.000041, 10, 0.00001)
plt.show()

In [31]:
FD1x

array([             nan,   3.81563561e-05,   2.06880812e-05,
         9.78593489e-06,   1.13328935e-05,   7.46311571e-06,
         5.99327100e-06,   6.87303673e-06,   6.13714680e-06,
         7.22939043e-06,   6.61574815e-06,   6.66538738e-06,
         7.44275402e-06,   6.19920578e-06,   5.01932609e-06,
         6.13612876e-06,   6.31862204e-06,   5.85145576e-06,
         4.34197759e-06,   4.99907318e-06,   5.28960029e-06,
         6.53276705e-06,   5.80062270e-06,   5.29881411e-06,
         7.35269750e-06,   5.03132025e-06,   4.65570184e-06,
         7.32538004e-06,   7.03853885e-06,   5.67497649e-06,
         6.99479012e-06,   8.10131901e-06,   7.27155389e-06,
         8.69119152e-06,   7.42849986e-06,   4.86708689e-06,
         7.30139234e-06,   6.65622891e-06,   7.09088433e-06,
         6.71971930e-06,   6.70527147e-06,   6.08426593e-06,
         5.98447845e-06,   7.10390178e-06,   6.72020401e-06,
         7.46473544e-06,   5.78164506e-06,   7.23004513e-06,
         6.20875353e-06,

In [32]:
def plot_histogram(M, time, total, datatype, filename, tn):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """
    total1 = total
    MSD = M
#     # Creates an array 'particles' that contains the particle number at each frame.
#     particles = traj[:, n1]
#     total = int(max(particles))
#     total1 = total + 1
#     rawtime = traj[:, n2]
#     bow = traj.shape[0]
#     raw2DMSDs = np.zeros((bow, 4))
#     raw2DMSDs[:, 0] = traj[:, n3]
#     raw2DMSDs[:, 1:4] = traj[:, n3 + 3:n3 + 6]
#     MSD = dict()
#     time = dict()

#     # Creates an array for each trajectory containing all xyz data
#     for num in range(1, total1):

#         hold = np.where(particles == num)
#         itindex = hold[0]
#         min1 = min(itindex)
#         max1 = max(itindex)
#         MSD[num] = (raw2DMSDs[min1+2:max1, :])
#         time[num] = (rawtime[min1+2:max1])
    
#     t = time[1][:]
    def find_nearest(array, value):
        idx = (np.abs(array-value)).argmin()
        return array[idx], idx

    td, idx = find_nearest(t, tn)
    
    hist = np.zeros(total1)
    for num in range(1, total1):
        hist[num] = MSD[num][idx, 0]
    
    plot, bins = np.histogram(hist, bins=20)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:])/2
    plt.bar(center, plot, align='center', width=width)
    plt.xlabel('D (um2/s)', fontsize=20)

    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return hist, MSD, total1

In [34]:
I[10]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71], dtype=int64)